<a href="https://colab.research.google.com/github/GenkiAkura/ProjekCV/blob/main/UTS_IbnuRezki_Fahrezii_4TIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics flask pyngrok opencv-python-headless --quiet
# Menginstall pustaka Python yang diperlukan seperti ultralytics, flask, pyngrok, dan opencv-python-headless menggunakan pip, dengan flag --quiet untuk menyembunyikan output detail.

from ultralytics import YOLO
# Mengimpor kelas YOLO dari pustaka ultralytics untuk penggunaan model deteksi objek.

import cv2
# Mengimpor pustaka OpenCV (cv2) untuk pemrosesan gambar dan video.

import numpy as np
# Mengimpor pustaka NumPy sebagai np untuk operasi numerik dan array.

import os
# Mengimpor pustaka os untuk berinteraksi dengan sistem operasi, seperti mengelola file dan direktori.

from pyngrok import ngrok
# Mengimpor modul ngrok dari pustaka pyngrok untuk membuat terowongan aman ke aplikasi lokal.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralyti

In [ ]:
!pip install flask pyngrok --quiet


In [2]:
# Fungsi untuk memproses gambar
def process_image(image_path, output_path):
    # Membaca gambar dari path yang diberikan menggunakan OpenCV
    img = cv2.imread(image_path)
    # Mengonversi warna gambar dari BGR ke RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Melakukan deteksi objek pada gambar menggunakan model
    results = model(image_path)

    # Iterasi melalui setiap hasil deteksi
    for result in results:
        boxes = result.boxes
        # Iterasi melalui setiap kotak deteksi
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            # Mendapatkan nilai kepercayaan (confidence) dari deteksi
            confidence = box.conf[0].item()
            # Mendapatkan ID kelas dari deteksi
            class_id = int(box.cls[0].item())
            # Mendapatkan label berdasarkan ID kelas dari model
            label = model.names[class_id]

            # Memeriksa jika label adalah 'bird'
            if label == 'bird':
                # Menggambar kotak persegi panjang di sekitar objek yang terdeteksi
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Menambahkan teks label dan kepercayaan di atas kotak
                cv2.putText(img, f'{label} {confidence:.2f}', (x1, y1 - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Menyimpan gambar yang telah diproses ke path output dengan konversi kembali ke BGR
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    # Mengembalikan path gambar yang telah diproses
    return output_path

In [3]:
# Fungsi untuk memproses video
def process_video(video_path, output_path):
    # Membuka video dari path yang diberikan menggunakan OpenCV
    cap = cv2.VideoCapture(video_path)
    # Mendapatkan lebar frame dari video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # Mendapatkan tinggi frame dari video
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # Mendapatkan frame rate (FPS) dari video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Mengatur codec video untuk output (mp4v)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # Membuat objek VideoWriter untuk menyimpan video yang telah diproses
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Memproses video frame demi frame selama video masih terbuka
    while cap.isOpened():
        ret, frame = cap.read()
        # Menghentikan loop jika tidak ada frame lagi
        if not ret:
            break

        # Melakukan deteksi objek pada frame menggunakan model
        results = model(frame)

        # Iterasi melalui setiap hasil deteksi
        for r in results:
            for box in r.boxes:
                cls = int(box.cls[0])
                # Mendapatkan label berdasarkan ID kelas dari model
                label = model.names[cls]
                # Memeriksa jika label adalah 'bird'
                if label == 'bird':
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    # Menggambar kotak persegi panjang di sekitar objek yang terdeteksi
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    # Menambahkan teks label dan kepercayaan di atas kotak
                    cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Menulis frame yang telah diproses ke file output
        out.write(frame)

    # Melepaskan objek VideoCapture
    cap.release()
    # Melepaskan objek VideoWriter
    out.release()
    # Mengembalikan path video yang telah diproses
    return output_path

In [4]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


In [7]:
from flask import Flask, request, render_template_string, send_file
import os
from threading import Thread
from ultralytics import YOLO

app = Flask(__name__)

# Load model YOLOv8
model = YOLO("yolov8n.pt")  # Gunakan model custom jika ada

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>YOLOv8 Bird Detector</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body {
            background-color: #f8f9fa;
            font-family: 'Arial', sans-serif;
        }
        .container {
            max-width: 800px;
            margin-top: 50px;
        }
        .card {
            border-radius: 15px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            padding: 20px;
        }
        .btn-custom {
            background-color: #007bff;
            color: white;
            transition: background-color 0.3s;
        }
        .btn-custom:hover {
            background-color: #0056b3;
        }
        .error-message {
            color: #dc3545;
            font-weight: bold;
        }
        .output-media {
            max-width: 100%;
            height: auto;
            border-radius: 10px;
            margin-top: 20px;
        }
        .download-link {
            margin-top: 15px;
            display: inline-block;
            text-decoration: none;
            color: #007bff;
            font-weight: bold;
        }
        .download-link:hover {
            text-decoration: underline;
        }
        @media (max-width: 576px) {
            .container {
                margin-top: 20px;
                padding: 15px;
            }
            .card {
                padding: 15px;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="card">
            <h2 class="text-center mb-4">YOLOv8 Bird Detector</h2>
            <form method="post" enctype="multipart/form-data">
                <div class="mb-3">
                    <label for="fileInput" class="form-label">Upload Image or Video</label>
                    <input type="file" class="form-control" id="fileInput" name="file" accept=".jpg,.jpeg,.png,.mp4">
                </div>
                <button type="submit" class="btn btn-custom w-100">Detect Birds</button>
            </form>

            {% if error %}
                <p class="error-message text-center mt-3">{{ error }}</p>
            {% endif %}

            {% if output_path %}
                <div class="text-center">
                    {% if is_image %}
                        <img src="{{ output_path }}" class="output-media" alt="Detection Result">
                    {% else %}
                        <video class="output-media" controls>
                            <source src="{{ output_path }}" type="video/mp4">
                            Your browser does not support the video tag.
                        </video>
                    {% endif %}
                    <p><a href="{{ output_path }}" class="download-link" download>Download Result</a></p>
                </div>
            {% endif %}
        </div>
    </div>
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
"""

def process_image(input_path, output_path):
    # Hanya deteksi class 'bird' (COCO ID 14)
    results = model(input_path, classes=[14])

    # Cek apakah burung terdeteksi
    if results[0].boxes is None or len(results[0].boxes.cls) == 0:
        raise ValueError("Tidak ada burung yang terdeteksi.")

    # Simpan hasil deteksi ke file
    results[0].save(filename=output_path)

def process_video(input_path, output_path):
    # Deteksi hanya 'bird' class (class_id 14)
    results = model.predict(source=input_path, save=True, classes=[14])

    # Cari file hasil dari folder YOLO
    pred_dir = 'runs/detect/predict'
    for file in os.listdir(pred_dir):
        if file.endswith(".mp4"):
            os.rename(os.path.join(pred_dir, file), output_path)
            break
    else:
        raise ValueError("Tidak ada burung yang terdeteksi di video.")

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        file = request.files.get('file')
        if not file or file.filename == '':
            return render_template_string(HTML_TEMPLATE, error="File tidak ditemukan")
        ext = file.filename.lower().split('.')[-1]
        if ext not in ['jpg', 'jpeg', 'png', 'mp4']:
            return render_template_string(HTML_TEMPLATE, error="Format tidak didukung")

        # Buat folder jika belum ada
        os.makedirs('static/uploads', exist_ok=True)
        os.makedirs('static/outputs', exist_ok=True)

        input_path = f'static/uploads/{file.filename}'
        output_path = f'static/outputs/output_{file.filename}'
        file.save(input_path)
        is_image = ext in ['jpg', 'jpeg', 'png']

        try:
            if is_image:
                process_image(input_path, output_path)
            else:
                process_video(input_path, output_path)
        except Exception as e:
            return render_template_string(HTML_TEMPLATE, error=f"Error: {str(e)}")

        return render_template_string(HTML_TEMPLATE, output_path='/' + output_path, is_image=is_image)

    return render_template_string(HTML_TEMPLATE)

@app.route('/static/<path:filename>')
def serve_static(filename):
    return send_file(os.path.join('static', filename))

# Jalankan Flask
def run():
    app.run(host='0.0.0.0', port=5000)

if __name__ == '__main__':
    Thread(target=run).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
!./cloudflared tunnel --url http://localhost:5000


2025-07-03T04:50:40Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-07-03T04:50:40Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-07-03T04:50:44Z INF +--------------------------------------------------------------------------------------------+
2025-07-03T04:50:44Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-07-03T04:50:44Z INF |  https://actions-realm-acrylic-approved.trycloudflare.

INFO:werkzeug:127.0.0.1 - - [03/Jul/2025 04:50:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jul/2025 04:50:56] "GET /favicon.ico HTTP/1.1" 404 -



image 1/1 /content/static/uploads/burung dan sapi.jpg: 448x640 1 bird, 58.1ms
Speed: 3.7ms preprocess, 58.1ms inference, 2.1ms postprocess per image at shape (1, 3, 448, 640)


INFO:werkzeug:127.0.0.1 - - [03/Jul/2025 04:51:07] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jul/2025 04:51:08] "GET /static/outputs/output_burung%20dan%20sapi.jpg HTTP/1.1" 200 -



image 1/1 /content/static/uploads/burungsapiii2.jpg: 512x640 1 bird, 41.6ms
Speed: 3.5ms preprocess, 41.6ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 640)


INFO:werkzeug:127.0.0.1 - - [03/Jul/2025 04:54:18] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jul/2025 04:54:18] "GET /static/outputs/output_burungsapiii2.jpg HTTP/1.1" 200 -
